In [93]:
import numpy as np
import pandas as pd
import os


In [94]:


# Load data to panda data frame
rows=[]
with open(os.path.join('dontpatronizeme_pcl.tsv')) as f:
    for line in f.readlines()[4:]:
        par_id=line.strip().split('\t')[0]
        art_id = line.strip().split('\t')[1]
        keyword=line.strip().split('\t')[2]
        country=line.strip().split('\t')[3]
        t=line.strip().split('\t')[4]#.lower()
        l=line.strip().split('\t')[-1]
        if l=='0' or l=='1':
            lbin=0
        else:
            lbin=1
        rows.append(
            {'par_id':par_id,
            'art_id':art_id,
            'keyword':keyword,
            'country':country,
            'text':t, 
            'label':lbin, 
            'orig_label':l
            }
            )
df=pd.DataFrame(rows, columns=['par_id', 'art_id', 'keyword', 'country', 'text', 'label', 'orig_label']) 
print(df.shape[0])
print(df[df['label'] == 0].shape[0])
print(df[df['label'] == 1].shape[0])


10469
9476
993


In [95]:

# Filter rows with a specific label (e.g., 'negative')
noPCL = df[df['label'] == 0]
PCL = df[df['label'] == 1]
average = df

# Get sentence length (in words)
noPCL['text_Length'] = noPCL['text'].apply(lambda x: len(x.split()))
PCL['text_Length'] = PCL['text'].apply(lambda x: len(x.split()))
average['text_length']  = df['text'].apply(lambda x: len(x.split()))

# Calculate average length
average_length_noPCL = noPCL['text_Length'].mean()
average_length_PCL = PCL['text_Length'].mean()
average_l = average['text_length'].mean()

print(average_length_noPCL)
print(average_length_PCL)
print(average_l)


47.87779653862389
53.620342396777446
48.42248543318369


/tmp/ipykernel_548156/4254412181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noPCL['text_Length'] = noPCL['text'].apply(lambda x: len(x.split()))
/tmp/ipykernel_548156/4254412181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PCL['text_Length'] = PCL['text'].apply(lambda x: len(x.split()))


In [96]:
# Get unique values from a column
total = df[df['keyword']=='vulnerable'].shape[0]
unique_keywords_noPCL = noPCL[noPCL['keyword']=='vulnerable'].shape[0]
unique_keywords_PCL = PCL[PCL['keyword']=='vulnerable'].shape[0]

print((unique_keywords_noPCL/total)*(993))
print((unique_keywords_PCL/total)*(9476))


919.4444444444445
701.9259259259259


In [97]:
# Split in to dev set and test set
train_ids = pd.read_csv('train_semeval_parids-labels.csv')
dev_ids = pd.read_csv('dev_semeval_parids-labels.csv')
train_ids = train_ids.iloc[:, 0].astype(str).tolist()
dev_ids = dev_ids.iloc[:, 0].astype(str).tolist()

train_df = df[df['par_id'].isin(train_ids)]
dev_df = df[df['par_id'].isin(dev_ids)]
print(len(train_df)+len(dev_df))

# Load the final test set
rows=[]
with open(os.path.join('task4_test.tsv')) as f:
    for line in f.readlines():
        par_id=line.strip().split('\t')[0]
        art_id = line.strip().split('\t')[1]
        keyword=line.strip().split('\t')[2]
        country=line.strip().split('\t')[3]
        rows.append(
            {'par_id':par_id,
            'art_id':art_id,
            'keyword':keyword,
            'country':country,
            'text':t, 
            }
            )
test_df=pd.DataFrame(rows, columns=['par_id', 'art_id', 'keyword', 'country', 'text']) 
print(test_df.shape[0])

10469
3832
